# Creating a Rock Paper Scissor Game

## 1. Load Required Libraries

In [1]:
import keras
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten , Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

import cv2
import copy
import time

Using TensorFlow backend.


## 2. Prepare data 

In [2]:
train_dir = "rps/rps"
test_dir = "rps-test-set/rps-test-set"

train_rock = os.listdir(train_dir +"/rock")
train_paper = os.listdir(train_dir +"/paper")
train_scissors = os.listdir(train_dir +"/scissors")

test_rock = os.listdir(train_dir +"/rock")
test_scissors = os.listdir(train_dir +"/scissors")
test_paper = os.listdir(train_dir +"/paper")


In [3]:
print("Number of images in the train-set:", len(train_rock) + len(train_paper) + len(train_scissors))
print("Number of images in the test-set:", len(test_rock) + len(test_paper) + len(test_scissors))

print("\nNumber of rocks in the train-set:", len(train_rock))
print("Number of papers in the train-set:", len(train_paper))
print("Number of scissors in the train-set:", len(train_scissors))

print("\nNumber of rocks in the test-set:", len(test_rock))
print("Number of papers in the test-set:", len(test_paper))
print("Number of scissors in the test-set:", len(test_scissors))

Number of images in the train-set: 2520
Number of images in the test-set: 2520

Number of rocks in the train-set: 840
Number of papers in the train-set: 840
Number of scissors in the train-set: 840

Number of rocks in the test-set: 840
Number of papers in the test-set: 840
Number of scissors in the test-set: 840


### Image Augmentation using Keras

In [4]:
train_datagen= ImageDataGenerator(rescale= 1/255.0,
                           width_shift_range =0.2 ,
                           height_shift_range=0.2 ,
                           zoom_range=0.2 ,
                           rotation_range=40 ,
                           shear_range=0.2, 
                           horizontal_flip= True)

test_datagen = ImageDataGenerator(rescale= 1/255.0)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(75,75), class_mode="categorical", batch_size=128)

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(75,75), class_mode="categorical", batch_size=128)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


## 3. Build Convolution Neural Net Model

In [5]:
model = Sequential()
model.add(Conv2D(64, (3,3), activation="relu", input_shape=(75,75,3), padding="same"))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(64, (3,3), activation="relu", padding="same"))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(128, (3,3), activation="relu", padding="same"))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(128, (3,3), activation="relu", padding="same"))
model.add(MaxPool2D((2,2)))

model.add(Flatten())

model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(512, activation= "relu"))
model.add(Dropout(0.2))

model.add(Dense(3, activation="softmax"))


model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 75, 75, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 9, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 9, 9, 128)        

In [6]:
## Create list of keras callbacks

my_callbacks_es = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
my_callbacks_rlr = ReduceLROnPlateau(monitor='val_accuracy', pateince=2, factor=0.5, min_lr=0.00001, verbose=1)
my_callbacks_mc = ModelCheckpoint("model.h5",monitor='val_accuracy', save_best_only=True, verbose=1, mode='max')
my_callbacks = [my_callbacks_es, my_callbacks_rlr, my_callbacks_mc]

In [7]:
## compile model

model.compile(loss= "categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [8]:
## train model

history= model.fit(train_generator, validation_data=test_generator, 
                  epochs = 25, verbose=1, steps_per_epoch= 20, validation_steps=3,
                  callbacks= my_callbacks)

Epoch 1/25
20/20 [==============================] - 117s 6s/step - loss: 1.1102 - accuracy: 0.3385 - val_loss: 1.0988 - val_accuracy: 0.3333

Epoch 00001: val_accuracy improved from -inf to 0.33333, saving model to model.h5
Epoch 2/25
20/20 [==============================] - 68s 3s/step - loss: 1.0909 - accuracy: 0.3821 - val_loss: 1.0193 - val_accuracy: 0.4435

Epoch 00002: val_accuracy improved from 0.33333 to 0.44355, saving model to model.h5
Epoch 3/25
20/20 [==============================] - 68s 3s/step - loss: 0.9180 - accuracy: 0.5361 - val_loss: 0.7387 - val_accuracy: 0.8038

Epoch 00003: val_accuracy improved from 0.44355 to 0.80376, saving model to model.h5
Epoch 4/25
20/20 [==============================] - 67s 3s/step - loss: 0.7174 - accuracy: 0.6405 - val_loss: 0.4374 - val_accuracy: 0.8333

Epoch 00004: val_accuracy improved from 0.80376 to 0.83333, saving model to model.h5
Epoch 5/25
20/20 [==============================] - 69s 3s/step - loss: 0.5704 - accuracy: 0.7321 

In [2]:
## load our trained model 

from keras.models import load_model
model = load_model("model.h5")

## 4. Using OpenCV to test model with real time video from webcam.

In [ ]:
## start webcam

cap = cv2.VideoCapture(0)

start = True

while True:
    
    ret, frame = cap.read()
    img =  copy.deepcopy(frame)
    
    cv2.rectangle(img, (20,50), (320, 350), (255,0,0), 3)
    
    roi = frame[40:340, 10:310] 
    
    input_img = cv2.resize(roi, (75,75))
    
    
    input_img = np.expand_dims(input_img, axis=0)
    list1 = ['PAPER', "ROCK", "SCISSORS"]
    array= model.predict(input_img)
    id= np.argmax(array)
    text = list1[id]
    cv2.putText(img, "Put your hand in the box", (20,30), cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,255), 2)
    cv2.putText(img, text, (20,380), cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,255,255), 2)
    cv2.imshow("frame", img)
    key = cv2.waitKey(1) & 0xFF
    if key== ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
